## 1) Sử dụng thư viện Spotipy:


In [1]:
import pandas as pd

In [ ]:
!pip install spotipy

In [ ]:
import spotipy
import sys
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

CLIENT_ID = '01df5da9702a44dd9e8ccfc0746055e7'
CLIENT_SECRET = 'de0273d1004843b683fd8053799bbb5a'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET))

In [ ]:
def get_random_search():
    characters = 'abcdefghijklmnopqrstuvwxyz'
    random_character = random.choice(characters)
    random_search = ''
    random_search = '%' + random_character + '%'
    return random_search

tracks = []
for i in range(0,1000):
  tracks.append(sp.search(get_random_search(),offset= random.randint(0, 500),limit=50))

In [ ]:
track_ids = list(track_df.id.unique())

In [ ]:
track_df = track_df[track_df.id.isin(track_ids)]

In [ ]:
features_df = []
for i in range(0,len(track_ids),50):
  features_df.append(sp.audio_features(track_ids[i:i+50]))

In [ ]:
flattened_list = [item for sublist in features_df for item in sublist]
filtered_data_list = [data for data in flattened_list if data is not None and data]
features_df = pd.DataFrame(filtered_data_list)

In [ ]:
features_df.to_csv('features_df.csv')

In [2]:
features_df = pd.read_csv("features_df.csv")

In [ ]:
popularity = []
song_name = []
artist_name = []
for id in features_df['id']:
  track = sp.track(id)
  artist_name.append(track['artists'][0]['name'])
  song_name.append(track['name'])
  popularity.append(track['popularity'])

In [ ]:
features_df['song_name'] = song_name
features_df['popularity'] = popularity
features_df['artist_name'] = artist_name

## 2) Sử dụng Genius để tìm id và release_date từng track:

In [3]:
import requests
import json

def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + '5l4zx-dfAx4KV5iNuGUQs2QJ_YsaVqIY_eStOF_CJLIISlt8jZcZkPaQmGCyES2_'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, params=data, headers=headers)

    return response

In [ ]:
def genius_id(song,artist):
  response = request_song_info(song, artist)
  if response.status_code == 200 :
    json = response.json()
    if(json['response']['hits'] != []):
      return json['response']['hits'][0]['result']['id']
  return -1

In [9]:
##add them genius url:
url = []
count = 0
for song, artist in zip(features_df['song_name'], features_df['artist_name']):
  response = request_song_info(song, artist)
  if response.status_code == 200 :
    json = response.json()
    if(json['response']['hits'] != []):
      url.append(json['response']['hits'][0]['result']['url'])
    else:
      url.append('-1')
  else:
    url.append('-1')
  print(count + 1)
  count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
features_df['genius_url'] = url

In [ ]:
id = []
date = []
count = 0
for song, artist in zip(features_df['song_name'], features_df['artist_name']):
  response = request_song_info(song, artist)
  if response.status_code == 200 :
    json = response.json()
    if(json['response']['hits'] != []):
      id.append(json['response']['hits'][0]['result']['id'])
      date.append(json['response']['hits'][0]['result']['release_date_components'])
    else:
      id.append('-1')
      date.append({'year': 0, 'month': 0, 'day': 0})
  else:
    id.append('-1')
    date.append({'year': 0, 'month': 0, 'day': 0})
  print(count + 1)
  count+=1

In [ ]:
features_df['genius_id'] = id
features_df['release_date'] = date

# 3) Cào tags, lyrics, tier:

In [ ]:
df = features_df

## Tu 0 den 399

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "8b7b8677cbmshda7d2be90521933p12dbbejsnf58fb3942d5b",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[:100]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_100.csv',index = False)

100 toi 200

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "fa8d2e20bemsh0425391cf102695p16716bjsn007e194a764e",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[100:200]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_200.csv',index = False)

200 toi 300

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "eb14d2c446mshe73641800cba6a7p159db1jsn5c0df1180378",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[200:300]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_300.csv',index = False)

300 toi 400

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "d942db774bmshd921792eea11f3ep178f38jsn01d246d8f41f",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[300:400]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  429
O


In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_400.csv',index = False)

## Tu 400 den 799


### 400 toi 499

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": '6bff30b75emshdd298c47c1c929ap1383f0jsn0c21e6195f9b',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[400:500]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  404
F
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_500.csv',index = False)

### Tu 500 den 599

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": '6f8d2ed624msh3e15216469ca746p12cc7ajsn072c037005c7',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[500:600]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_600.csv',index = False)

### Tu 600 den 699

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": 'bbf5f5b11emshaf127affd902405p12fe5ajsn829419d7abe6',
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[600:700]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  404
F
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  404
F
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_700.csv',index = False)

### Tu 700 den 799

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "01cb255682mshe8a5b984adc38c4p168ac3jsnd76139a04888",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[700:800]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_800.csv',index = False)

In [ ]:
concat_df = pd.read_csv('df_180.csv', encoding = 'utf-8')

In [ ]:
df_500 = pd.read_csv('df_500.csv', encoding = 'utf-8')
df_600 = pd.read_csv('df_600.csv', encoding = 'utf-8')
df_700 = pd.read_csv('df_700.csv', encoding = 'utf-8')
df_800 = pd.read_csv('df_800.csv', encoding = 'utf-8')


In [ ]:
full = pd.concat([df_500,df_600,df_700,df_800],axis=0)

In [ ]:
full.to_csv('df_400_799.csv', index = False)

## Tu 800 den 1199

### 800 900

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "9f5fe71373msh277bfa090fa9e3fp19c725jsn65170baf2bc9",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[800:900]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  404
F
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_900.csv',index = False)

### 900 toi 1000

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "80f5fe4f2bmshd2c8a43c16f29aep1e110fjsn1a258cb62e0f",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[900:1000]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1000.csv',index = False)

### 1000 toi 1100

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "b45e4843c0msh0f2c7367f96e63bp103ddajsnd1492df1a4ba",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1000:1100]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1100.csv',index = False)

### 1100 toi 1200

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "8d560cf78dmshafb63ed3984e7f0p11078ejsn14c9a26443a2",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1100:1200]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  404
F
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1200.csv',index = False)

## Tu 1200 den 1600

### 1200 1300

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "a701564729msh5f035b6bffd776ap1f4ff7jsncdf04de16691",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1200:1300]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  404
F
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1300.csv',index = False)

### 1300 toi 1400

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "f6162d6b19mshbb316394fafd2ddp10376fjsn74ba3f927013",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1300:1400]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  404
F
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1400.csv',index = False)

### 1400 toi 1500

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "b6af23bd98mshaaefcc5917a3b45p162425jsnd49637f22d70",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1400:1500]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  200
Y
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  200
Y
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  404
F
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1500.csv',index = False)

### 1500 toi 1600

In [ ]:
import requests
lyrics = []
tags = []
tier = []
url = "https://genius-song-lyrics1.p.rapidapi.com/song/lyrics/"
headers = {
	#Thay key cua tung acc vao day
  "X-RapidAPI-Key": "34ebec4defmsh5fdced7a2c4389bp1a8703jsnfab38bc588d5",
	"X-RapidAPI-Host": "genius-song-lyrics1.p.rapidapi.com"
}
count = 0
for id in df['genius_id'].iloc[1500:1600]:
    querystring = {"id":id}
    response = requests.get(url, headers=headers, params=querystring)
    print(count , ' | ',response.status_code)
    if response.status_code == 200:
      data = response.json()
      lyrics.append(data['lyrics']['lyrics']['body']['html'])
      tags.append(data['lyrics']['tracking_data']['primary_tag'])
      tier.append(data['lyrics']['tracking_data']['song_tier'])
      print('Y') #Yes : thanh cong
    elif response.status_code != 429:
      lyrics.append(0)
      tags.append(0)
      tier.append(0)
      print('F') #Fail, khong lay duoc
    else:
      print('O') #Het requests
      break
    count = count + 1

0  |  200
Y
1  |  404
F
2  |  200
Y
3  |  200
Y
4  |  200
Y
5  |  200
Y
6  |  404
F
7  |  200
Y
8  |  200
Y
9  |  200
Y
10  |  200
Y
11  |  200
Y
12  |  200
Y
13  |  200
Y
14  |  200
Y
15  |  200
Y
16  |  200
Y
17  |  200
Y
18  |  200
Y
19  |  200
Y
20  |  200
Y
21  |  200
Y
22  |  200
Y
23  |  200
Y
24  |  200
Y
25  |  200
Y
26  |  200
Y
27  |  200
Y
28  |  200
Y
29  |  200
Y
30  |  200
Y
31  |  200
Y
32  |  200
Y
33  |  200
Y
34  |  200
Y
35  |  200
Y
36  |  200
Y
37  |  200
Y
38  |  200
Y
39  |  200
Y
40  |  200
Y
41  |  200
Y
42  |  200
Y
43  |  200
Y
44  |  200
Y
45  |  200
Y
46  |  200
Y
47  |  200
Y
48  |  200
Y
49  |  200
Y
50  |  200
Y
51  |  200
Y
52  |  200
Y
53  |  200
Y
54  |  200
Y
55  |  200
Y
56  |  200
Y
57  |  200
Y
58  |  200
Y
59  |  200
Y
60  |  200
Y
61  |  200
Y
62  |  200
Y
63  |  200
Y
64  |  200
Y
65  |  200
Y
66  |  200
Y
67  |  200
Y
68  |  200
Y
69  |  200
Y
70  |  200
Y
71  |  200
Y
72  |  200
Y
73  |  200
Y
74  |  200
Y
75  |  200
Y
76  |  200
Y
77  |  20

In [ ]:
new_df = pd.DataFrame({
    'lyric' : lyrics,
    'tags': tags,
    'tier':tier
})
new_df.to_csv('df_1600.csv',index = False)

In [ ]:
df1300 = pd.read_csv("df_1300.csv",encoding = "utf-8")
df1400 = pd.read_csv("df_1400.csv",encoding = "utf-8")
df1500 = pd.read_csv("df_1500.csv",encoding = "utf-8")
df1600 = pd.read_csv("df_1600.csv",encoding = "utf-8")

In [ ]:
full = pd.concat([df1300,df1400,df1500,df1600],axis=0)

In [ ]:
full.to_csv('df_1200_1599.csv',index = False,encoding = "utf-8")